In [1]:
import os, sys, glob, re, time
from os.path import exists

import geopandas as gpd 
import pandas as pd
import numpy as np 
import fiona
from shapely.geometry import Point, LineString, Polygon, shape, MultiPoint
import shapely.wkt
from shapely.ops import cascaded_union

from xrspatial import zonal_stats 
import rasterio
from rasterio.plot import show
from rasterio import features, mask
from rasterio.features import shapes
from osgeo import gdal, osr, ogr, gdal_array

In [2]:
ProjectFolder = os.getcwd()
print(ProjectFolder)

C:\Users\wb527163\GEO_Cdrive_Grace\Sahel_Cdrive\exploratory_climate_study\Agriculture


### GAEZ

Before running the script: Downloaded the data for each year and added desired production data to respective year folders.

In [8]:
# Make a list of rainfed agricultural production rasters for each year.
GAEZ2010 = [i for i in os.listdir(r'GAEZ/') if i.endswith('.tif')]

print(GAEZ2010)

['ban_2010_prd.tif', 'brl_2010_prd.tif', 'cot_2010_prd.tif', 'grd_2010_prd.tif', 'mlt_2010_prd.tif', 'mze_2010_prd.tif', 'oce_2010_prd.tif', 'olp_2010_prd.tif', 'olv_2010_prd.tif', 'rcw_2010_prd.tif', 'rsd_2010_prd.tif', 'rt1_2010_prd.tif', 'rt2_2010_prd.tif', 'rt3_2010_prd.tif', 'sfl_2010_prd.tif', 'soy_2010_prd.tif', 'srg_2010_prd.tif', 'sub_2010_prd.tif', 'suc_2010_prd.tif', 'tob_2010_prd.tif', 'whe_2010_prd.tif']


In [9]:
def SimplifyFileNames(folder):
    for count, filename in enumerate(os.listdir(folder)):
            dst = ''.join(['Crop', str(count), '.tif'])
            src =f"{folder}/{filename}"  # foldername/filename, if .py file is outside folder
            dst =f"{folder}/{dst}"

            os.rename(src, dst)

In [11]:
SimplifyFileNames(folder = r'GAEZ/mln_GK')
SimplifyFileNames(folder = 'GAEZ')

In [ ]:
# Open command line for this:
# Note that I switched to my personal computer for this step. Gdal distribution wasn't getting recognized for some reason here.
gdal_create -if C:\Users\grace\GIS\povertyequity\exploratory_sahel\GAEZ\tons\Crop1.tif -burn 0 C:\Users\grace\GIS\povertyequity\exploratory_sahel\GAEZ\GAEZ_prodsum_1000t.tif
gdal_create -if C:\Users\grace\GIS\povertyequity\exploratory_sahel\GAEZ\mln_GK\Crop1.tif -burn 0 C:\Users\grace\GIS\povertyequity\exploratory_sahel\GAEZ\GAEZ_prodsum_mlnGK.tif

gdal_calc.py -A C:\Users\grace\GIS\povertyequity\exploratory_sahel\GAEZ\tons\*.tif --outfile=C:\Users\grace\GIS\povertyequity\exploratory_sahel\GAEZ\GAEZ_prodsum_1000t.tif --calc="numpy.sum(A,axis=0)"
gdal_calc.py -A C:\Users\grace\GIS\povertyequity\exploratory_sahel\GAEZ\mln_GK\*.tif --outfile=C:\Users\grace\GIS\povertyequity\exploratory_sahel\GAEZ\GAEZ_prodsum_mlnGK.tif --calc="numpy.sum(A,axis=0)"


#### Then in Arc, I did these steps for each of the 4 datasets:
- Raster calculator:  Int("ProdSum1000t.tif" * 100) = ProdSum1000t_int100 (mask: Sahel3countries_10kmBuffer)
- Raster to Polygon: ProdSum1000t_int100 to ProdSum1000t_int100_ply (No simplify, no multipart, CRS = Africa Albers)
- Delete rows where gridcode <= 0
- ProdSum1000t_int100_ply['ProdSumTons'] = float(!gridcode!)/100*(500/!Shape_Area!)

#### Spatial join: Input = PtGrids. Join layer = ProdSum2017_per500m_ply. How = within.

In [3]:
Spam00 = gpd.read_file("SPAM.gdb", layer = "ProdSum00_int100_ply")[['ProdSum00', 'geometry']].to_crs("ESRI:102022")
Spam05 = gpd.read_file("SPAM.gdb", layer = "ProdSum05_int100_ply")[['ProdSum05', 'geometry']].to_crs("ESRI:102022")
Spam10 = gpd.read_file("SPAM.gdb", layer = "ProdSum10_int100_ply")[['ProdSum10', 'geometry']].to_crs("ESRI:102022")
Spam17 = gpd.read_file("SPAM.gdb", layer = "ProdSum17_int100_ply")[['ProdSum17', 'geometry']].to_crs("ESRI:102022")

In [4]:
GridsFolder = r'C:\Users\wb527163\GEO_Cdrive_Grace\Sahel_Cdrive\exploratory_climate_study\grid_groupings'
GridPts = [i for i in os.listdir(GridsFolder) if 
           i.endswith('.shp') and 
           os.path.isfile(os.path.join(GridsFolder, i))]
GridPts

['grid_BFA01.shp',
 'grid_BFA02.shp',
 'grid_MLI01.shp',
 'grid_MLI02.shp',
 'grid_MLI03.shp',
 'grid_MLI04.shp',
 'grid_MLI05.shp',
 'grid_MLI06.shp',
 'grid_MLI07.shp',
 'grid_NER01.shp',
 'grid_NER02.shp',
 'grid_NER03.shp',
 'grid_NER04.shp',
 'grid_NER05.shp']

In [14]:
Spam00.head()

,ProdSum00,geometry
0,0.000027,"MULTIPOLYGON (((-2646881.723 1958446.738, -265..."
1,0.000222,"MULTIPOLYGON (((-2646881.723 1949089.252, -265..."
2,0.000098,"MULTIPOLYGON (((-2637524.237 1949089.252, -264..."
3,0.000003,"MULTIPOLYGON (((-2525234.407 1930374.280, -253..."
4,0.000004,"MULTIPOLYGON (((-2637524.237 1921016.795, -263..."


In [6]:
CheckContents = gpd.read_file(os.path.join(GridsFolder, GridPts[1]))
CheckContents

,ORIG_FID,admin1Pcod,GRID_ID,geometry
0,2928,BF56,2928BF56,POINT (-0.67637 15.08279)
1,2929,BF56,2929BF56,POINT (-0.67159 15.08274)
2,2930,BF56,2930BF56,POINT (-0.66680 15.08269)
3,2931,BF56,2931BF56,POINT (-0.66201 15.08264)
4,2932,BF56,2932BF56,POINT (-0.65722 15.08259)
...,...,...,...,...
741531,1070216,BF48,1070216BF48,POINT (-0.63196 10.81038)
741532,1070217,BF48,1070217BF48,POINT (-0.62716 10.81033)
741533,1070218,BF48,1070218BF48,POINT (-0.62236 10.81028)
741534,1070219,BF48,1070219BF48,POINT (-0.61756 10.81024)


In [7]:
del CheckContents

In [5]:
for item in GridPts:
    GridName = item[5:-4] # Drop the first 5 and last 4 characters of the file name to get the grouping name.
    PtGroup = gpd.read_file(os.path.join(GridsFolder, item))[['GRID_ID', 'geometry']].to_crs("ESRI:102022")
    print('Loaded gridded points for grouping %s. %s \n' % (GridName, time.ctime()))
    
    SpamPt = gpd.sjoin(
        PtGroup, 
        Spam00, how='left', predicate='within', lsuffix='pt', rsuffix='ag')
    print('\nJoined MapSPAM 2000 data onto grid ID for grouping %s. %s \n' % (GridName, time.ctime()))

    SpamPt = gpd.sjoin(
        SpamPt[['GRID_ID', 'ProdSum00', 'geometry']], 
        Spam05, how='left', predicate='within', lsuffix='pt', rsuffix='ag')
    print('\nJoined MapSPAM 2005 data onto grid ID for grouping %s. %s \n' % (GridName, time.ctime()))

    SpamPt = gpd.sjoin(
        SpamPt[['GRID_ID', 'ProdSum00', 'ProdSum05', 'geometry']], 
        Spam10, how='left', predicate='within', lsuffix='pt', rsuffix='ag')
    print('\nJoined MapSPAM 2010 data onto grid ID for grouping %s. %s \n' % (GridName, time.ctime()))

    SpamPt = gpd.sjoin(
        SpamPt[['GRID_ID', 'ProdSum00', 'ProdSum05', 'ProdSum10', 'geometry']], 
        Spam17, how='left', predicate='within', lsuffix='pt', rsuffix='ag')
    print('\nJoined MapSPAM 2017 data onto grid ID for grouping %s. %s \n' % (GridName, time.ctime()))
      
    print(SpamPt.sample(10))
    
    SpamPt = pd.DataFrame(SpamPt)[['GRID_ID', 'ProdSum00', 'ProdSum05', 'ProdSum10', 'ProdSum17']]
    print('\nDropped geometry for grouping %s. %s \n' % (GridName, time.ctime()))
    
    SpamPt.to_csv(''.join(['SPAM_', GridName, '.csv']))
    print('\nExported as table, grouping %s. %s \n' % (GridName, time.ctime()))
    del GridName, PtGroup, SpamPt

print('\n \n Finished assigning grid-level agricultural production data onto gridded points, all years. %s' % time.ctime())

Loaded gridded points for grouping BFA01. Mon Nov 21 13:40:36 2022 


Joined MapSPAM 2000 data onto grid ID for grouping BFA01. Mon Nov 21 13:40:59 2022 


Joined MapSPAM 2005 data onto grid ID for grouping BFA01. Mon Nov 21 13:41:20 2022 


Joined MapSPAM 2010 data onto grid ID for grouping BFA01. Mon Nov 21 13:41:42 2022 


Joined MapSPAM 2017 data onto grid ID for grouping BFA01. Mon Nov 21 13:42:05 2022 

            GRID_ID  ProdSum00  ProdSum05  ProdSum10  \
385295  1202669BF57        NaN   0.002280   0.005283   
375867  1193241BF57   0.000040        NaN        NaN   
370039  1187413BF57        NaN   0.001728   0.000840   
243053  1053519BF57   0.006411   0.009774   0.011962   
50156    422723BF46   0.001357        NaN   0.001682   
83468    561675BF46   0.003489        NaN   0.004288   
287103  1103261BF47        NaN        NaN   0.002222   
64361    481399BF46   0.008144        NaN   0.000999   
322189  1139563BF47   0.002523        NaN   0.001404   
88246    582940BF46   0.004


Exported as table, grouping MLI03. Mon Nov 21 14:00:53 2022 

Loaded gridded points for grouping MLI04. Mon Nov 21 14:05:59 2022 


Joined MapSPAM 2000 data onto grid ID for grouping MLI04. Mon Nov 21 14:07:13 2022 


Joined MapSPAM 2005 data onto grid ID for grouping MLI04. Mon Nov 21 14:08:32 2022 


Joined MapSPAM 2010 data onto grid ID for grouping MLI04. Mon Nov 21 14:09:47 2022 


Joined MapSPAM 2017 data onto grid ID for grouping MLI04. Mon Nov 21 14:11:04 2022 

             GRID_ID  ProdSum00  ProdSum05  ProdSum10  \
738762   3037379ML06        NaN        NaN        NaN   
721223   2991672ML06        NaN        NaN        NaN   
1456977  5401675ML06        NaN        NaN        NaN   
70548     612444ML06        NaN        NaN        NaN   
728970   3011149ML06        NaN        NaN        NaN   
658717   2823935ML06        NaN        NaN        NaN   
812424   3236169ML06        NaN        NaN        NaN   
1459181  5409272ML06        NaN        NaN        NaN   
87488     7


Exported as table, grouping NER01. Mon Nov 21 14:30:07 2022 

Loaded gridded points for grouping NER02. Mon Nov 21 14:32:14 2022 


Joined MapSPAM 2000 data onto grid ID for grouping NER02. Mon Nov 21 14:32:46 2022 


Joined MapSPAM 2005 data onto grid ID for grouping NER02. Mon Nov 21 14:33:16 2022 


Joined MapSPAM 2010 data onto grid ID for grouping NER02. Mon Nov 21 14:33:47 2022 


Joined MapSPAM 2017 data onto grid ID for grouping NER02. Mon Nov 21 14:34:18 2022 

            GRID_ID  ProdSum00  ProdSum05  ProdSum10  \
25706   5055273NE05        NaN        NaN        NaN   
284300  7055030NE05        NaN   0.001251   0.002945   
330325  7280130NE05        NaN        NaN        NaN   
373958  7487977NE05   0.002828        NaN   0.000005   
185776  6496720NE05        NaN        NaN   0.002205   
345824  7352360NE04   0.000111        NaN   0.006385   
402422  7623736NE05   0.000045   0.008100   0.010392   
447897  7838451NE05        NaN   0.002245   0.000959   
162935  6357488NE05 

### MapSPAM

Before running the script: Downloaded the data for each year and added desired production data to respective year folders.

In [7]:
# Make a list of rainfed, low-input agricultural production rasters for each year.
spam2000 = [i for i in os.listdir(r'spam2000_prod_rain_lowinput/') if i.endswith('_L.tif')]
spam2005 = [i for i in os.listdir(r'spam2005_prod_rain_lowinput/') if i.endswith('_L.tif')]
spam2010 = [i for i in os.listdir(r'spam2010_prod_rain_lowinput/') if i.endswith('_L.tif')]
spam2017 = [i for i in os.listdir(r'spam2017_prod_rain_lowinput/') if i.endswith('_L.tif')]

print(spam2000, spam2005, spam2010, spam2017)

['spam2000v3r7_production_BANP_L.tif', 'spam2000v3r7_production_BARL_L.tif', 'spam2000v3r7_production_BEAN_L.tif', 'spam2000v3r7_production_CASS_L.tif', 'spam2000v3r7_production_COFF_L.tif', 'spam2000v3r7_production_COTT_L.tif', 'spam2000v3r7_production_GROU_L.tif', 'spam2000v3r7_production_MAIZ_L.tif', 'spam2000v3r7_production_OFIB_L.tif', 'spam2000v3r7_production_OOIL_L.tif', 'spam2000v3r7_production_OPUL_L.tif', 'spam2000v3r7_production_OTHE_L.tif', 'spam2000v3r7_production_POTA_L.tif', 'spam2000v3r7_production_RICE_L.tif', 'spam2000v3r7_production_SORG_L.tif', 'spam2000v3r7_production_SOYB_L.tif', 'spam2000v3r7_production_SUGB_L.tif', 'spam2000v3r7_production_SUGC_L.tif', 'spam2000v3r7_production_SWPY_L.tif', 'spam2000v3r7_production_WHEA_L.tif'] ['SPAM2005V3r2_global_P_TL_ACOF_L.tif', 'SPAM2005V3r2_global_P_TL_BANA_L.tif', 'SPAM2005V3r2_global_P_TL_BARL_L.tif', 'SPAM2005V3r2_global_P_TL_BEAN_L.tif', 'SPAM2005V3r2_global_P_TL_CASS_L.tif', 'SPAM2005V3r2_global_P_TL_CHIC_L.tif', 'SPA

In [ ]:
CheckContents = [rasterio.open(r"spam2000_prod_rain_lowinput/spam2000v3r7_production_BANP_L.tif"), 
               rasterio.open(r"spam2005_prod_rain_lowinput/SPAM2005V3r2_global_P_TL_ACOF_L.tif"), 
               rasterio.open(r"spam2010_prod_rain_lowinput/spam2010V2r0_global_P_ACOF_L.tif"),
               rasterio.open(r"spam2017_prod_rain_lowinput/spam2017V2r1_SSA_P_ACOF_L.tif")]

for item in CheckContents:
    print(item.name, "\nBands= ", item.count, "\nWxH= ", item.width, "x", item.height, "\n\n")

stats = []
for item in CheckContents:
    band = item.read(1)
    stats.append({
        'raster': item.name,
        'min': band.min(),
        'mean': band.mean(),
        'median': np.median(band),
        'max': band.max()})

# Show stats for each channel
print("\n", stats)

In [20]:
CheckContents = None
band = None
item = None

In [17]:
def SimplifyFileNames(folder):
    for count, filename in enumerate(os.listdir(folder)):
            dst = ''.join(['Crop', str(count), '.tif'])
            src =f"{folder}/{filename}"  # foldername/filename, if .py file is outside folder
            dst =f"{folder}/{dst}"

            os.rename(src, dst)

In [21]:
SimplifyFileNames(folder = 'spam2000_prod_rain_lowinput')
SimplifyFileNames(folder = 'spam2005_prod_rain_lowinput')
SimplifyFileNames(folder = 'spam2010_prod_rain_lowinput')
SimplifyFileNames(folder = 'spam2017_prod_rain_lowinput')

In [ ]:
# Open command line for this:
# Note that I switched to my personal computer for this step. Gdal distribution wasn't getting recognized for some reason here.
gdal_create -if C:\Users\grace\GIS\povertyequity\exploratory_sahel\MapSPAM\spam2000_prod_rain_lowinput\Crop1.tif -burn 0 C:\Users\grace\GIS\povertyequity\exploratory_sahel\MapSPAM\ProdSum\ProdSum2000.tif
gdal_create -if C:\Users\grace\GIS\povertyequity\exploratory_sahel\MapSPAM\spam2005_prod_rain_lowinput\Crop1.tif -burn 0 C:\Users\grace\GIS\povertyequity\exploratory_sahel\MapSPAM\ProdSum\ProdSum2005.tif
gdal_create -if C:\Users\grace\GIS\povertyequity\exploratory_sahel\MapSPAM\spam2010_prod_rain_lowinput\Crop1.tif -burn 0 C:\Users\grace\GIS\povertyequity\exploratory_sahel\MapSPAM\ProdSum\ProdSum2010.tif
gdal_create -if C:\Users\grace\GIS\povertyequity\exploratory_sahel\MapSPAM\spam2017_prod_rain_lowinput\Crop1.tif -burn 0 C:\Users\grace\GIS\povertyequity\exploratory_sahel\MapSPAM\ProdSum\ProdSum2017.tif

gdal_calc.py -A C:\Users\grace\GIS\povertyequity\exploratory_sahel\MapSPAM\spam2000_prod_rain_lowinput\*.tif --outfile=C:\Users\grace\GIS\povertyequity\exploratory_sahel\MapSPAM\ProdSum\ProdSum2000.tif --calc="numpy.sum(A,axis=0)"
gdal_calc.py -A C:\Users\grace\GIS\povertyequity\exploratory_sahel\MapSPAM\spam2005_prod_rain_lowinput\*.tif --outfile=C:\Users\grace\GIS\povertyequity\exploratory_sahel\MapSPAM\ProdSum\ProdSum2005.tif --calc="numpy.sum(A,axis=0)"
gdal_calc.py -A C:\Users\grace\GIS\povertyequity\exploratory_sahel\MapSPAM\spam2010_prod_rain_lowinput\*.tif --outfile=C:\Users\grace\GIS\povertyequity\exploratory_sahel\MapSPAM\ProdSum\ProdSum2010.tif --calc="numpy.sum(A,axis=0)"
gdal_calc.py -A C:\Users\grace\GIS\povertyequity\exploratory_sahel\MapSPAM\spam2017_prod_rain_lowinput\*.tif --outfile=C:\Users\grace\GIS\povertyequity\exploratory_sahel\MapSPAM\ProdSum\ProdSum2017.tif --calc="numpy.sum(A,axis=0)"


#### Then in Arc, I did these steps for each of the 4 datasets:
- Raster calculator:  Int("ProdSum2017.tif" * 100) = ProdSum2017_int100 (mask: Sahel3countries_10kmBuffer)
- Raster to Polygon: ProdSum2017_int100 to ProdSum2017_int100_ply (No simplify, no multipart, CRS = Africa Albers)
- Delete rows where gridcode <= 0
- ProdSum17_int100_ply['ProdSum17'] = float(!gridcode!)/100*(500/!Shape_Area!)

#### Spatial join: Input = PtGrids. Join layer = ProdSum2017_per500m_ply. How = within.

In [3]:
Spam00 = gpd.read_file("SPAM.gdb", layer = "ProdSum00_int100_ply")[['ProdSum00', 'geometry']].to_crs("ESRI:102022")
Spam05 = gpd.read_file("SPAM.gdb", layer = "ProdSum05_int100_ply")[['ProdSum05', 'geometry']].to_crs("ESRI:102022")
Spam10 = gpd.read_file("SPAM.gdb", layer = "ProdSum10_int100_ply")[['ProdSum10', 'geometry']].to_crs("ESRI:102022")
Spam17 = gpd.read_file("SPAM.gdb", layer = "ProdSum17_int100_ply")[['ProdSum17', 'geometry']].to_crs("ESRI:102022")

In [4]:
GridsFolder = r'C:\Users\wb527163\GEO_Cdrive_Grace\Sahel_Cdrive\exploratory_climate_study\grid_groupings'
GridPts = [i for i in os.listdir(GridsFolder) if 
           i.endswith('.shp') and 
           os.path.isfile(os.path.join(GridsFolder, i))]
GridPts

['grid_BFA01.shp',
 'grid_BFA02.shp',
 'grid_MLI01.shp',
 'grid_MLI02.shp',
 'grid_MLI03.shp',
 'grid_MLI04.shp',
 'grid_MLI05.shp',
 'grid_MLI06.shp',
 'grid_MLI07.shp',
 'grid_NER01.shp',
 'grid_NER02.shp',
 'grid_NER03.shp',
 'grid_NER04.shp',
 'grid_NER05.shp']

In [14]:
Spam00.head()

,ProdSum00,geometry
0,0.000027,"MULTIPOLYGON (((-2646881.723 1958446.738, -265..."
1,0.000222,"MULTIPOLYGON (((-2646881.723 1949089.252, -265..."
2,0.000098,"MULTIPOLYGON (((-2637524.237 1949089.252, -264..."
3,0.000003,"MULTIPOLYGON (((-2525234.407 1930374.280, -253..."
4,0.000004,"MULTIPOLYGON (((-2637524.237 1921016.795, -263..."


In [6]:
CheckContents = gpd.read_file(os.path.join(GridsFolder, GridPts[1]))
CheckContents

,ORIG_FID,admin1Pcod,GRID_ID,geometry
0,2928,BF56,2928BF56,POINT (-0.67637 15.08279)
1,2929,BF56,2929BF56,POINT (-0.67159 15.08274)
2,2930,BF56,2930BF56,POINT (-0.66680 15.08269)
3,2931,BF56,2931BF56,POINT (-0.66201 15.08264)
4,2932,BF56,2932BF56,POINT (-0.65722 15.08259)
...,...,...,...,...
741531,1070216,BF48,1070216BF48,POINT (-0.63196 10.81038)
741532,1070217,BF48,1070217BF48,POINT (-0.62716 10.81033)
741533,1070218,BF48,1070218BF48,POINT (-0.62236 10.81028)
741534,1070219,BF48,1070219BF48,POINT (-0.61756 10.81024)


In [7]:
del CheckContents

In [5]:
for item in GridPts:
    GridName = item[5:-4] # Drop the first 5 and last 4 characters of the file name to get the grouping name.
    PtGroup = gpd.read_file(os.path.join(GridsFolder, item))[['GRID_ID', 'geometry']].to_crs("ESRI:102022")
    print('Loaded gridded points for grouping %s. %s \n' % (GridName, time.ctime()))
    
    SpamPt = gpd.sjoin(
        PtGroup, 
        Spam00, how='left', predicate='within', lsuffix='pt', rsuffix='ag')
    print('\nJoined MapSPAM 2000 data onto grid ID for grouping %s. %s \n' % (GridName, time.ctime()))

    SpamPt = gpd.sjoin(
        SpamPt[['GRID_ID', 'ProdSum00', 'geometry']], 
        Spam05, how='left', predicate='within', lsuffix='pt', rsuffix='ag')
    print('\nJoined MapSPAM 2005 data onto grid ID for grouping %s. %s \n' % (GridName, time.ctime()))

    SpamPt = gpd.sjoin(
        SpamPt[['GRID_ID', 'ProdSum00', 'ProdSum05', 'geometry']], 
        Spam10, how='left', predicate='within', lsuffix='pt', rsuffix='ag')
    print('\nJoined MapSPAM 2010 data onto grid ID for grouping %s. %s \n' % (GridName, time.ctime()))

    SpamPt = gpd.sjoin(
        SpamPt[['GRID_ID', 'ProdSum00', 'ProdSum05', 'ProdSum10', 'geometry']], 
        Spam17, how='left', predicate='within', lsuffix='pt', rsuffix='ag')
    print('\nJoined MapSPAM 2017 data onto grid ID for grouping %s. %s \n' % (GridName, time.ctime()))
      
    print(SpamPt.sample(10))
    
    SpamPt = pd.DataFrame(SpamPt)[['GRID_ID', 'ProdSum00', 'ProdSum05', 'ProdSum10', 'ProdSum17']]
    print('\nDropped geometry for grouping %s. %s \n' % (GridName, time.ctime()))
    
    SpamPt.to_csv(''.join(['SPAM_', GridName, '.csv']))
    print('\nExported as table, grouping %s. %s \n' % (GridName, time.ctime()))
    del GridName, PtGroup, SpamPt

print('\n \n Finished assigning grid-level agricultural production data onto gridded points, all years. %s' % time.ctime())

Loaded gridded points for grouping BFA01. Mon Nov 21 13:40:36 2022 


Joined MapSPAM 2000 data onto grid ID for grouping BFA01. Mon Nov 21 13:40:59 2022 


Joined MapSPAM 2005 data onto grid ID for grouping BFA01. Mon Nov 21 13:41:20 2022 


Joined MapSPAM 2010 data onto grid ID for grouping BFA01. Mon Nov 21 13:41:42 2022 


Joined MapSPAM 2017 data onto grid ID for grouping BFA01. Mon Nov 21 13:42:05 2022 

            GRID_ID  ProdSum00  ProdSum05  ProdSum10  \
385295  1202669BF57        NaN   0.002280   0.005283   
375867  1193241BF57   0.000040        NaN        NaN   
370039  1187413BF57        NaN   0.001728   0.000840   
243053  1053519BF57   0.006411   0.009774   0.011962   
50156    422723BF46   0.001357        NaN   0.001682   
83468    561675BF46   0.003489        NaN   0.004288   
287103  1103261BF47        NaN        NaN   0.002222   
64361    481399BF46   0.008144        NaN   0.000999   
322189  1139563BF47   0.002523        NaN   0.001404   
88246    582940BF46   0.004


Exported as table, grouping MLI03. Mon Nov 21 14:00:53 2022 

Loaded gridded points for grouping MLI04. Mon Nov 21 14:05:59 2022 


Joined MapSPAM 2000 data onto grid ID for grouping MLI04. Mon Nov 21 14:07:13 2022 


Joined MapSPAM 2005 data onto grid ID for grouping MLI04. Mon Nov 21 14:08:32 2022 


Joined MapSPAM 2010 data onto grid ID for grouping MLI04. Mon Nov 21 14:09:47 2022 


Joined MapSPAM 2017 data onto grid ID for grouping MLI04. Mon Nov 21 14:11:04 2022 

             GRID_ID  ProdSum00  ProdSum05  ProdSum10  \
738762   3037379ML06        NaN        NaN        NaN   
721223   2991672ML06        NaN        NaN        NaN   
1456977  5401675ML06        NaN        NaN        NaN   
70548     612444ML06        NaN        NaN        NaN   
728970   3011149ML06        NaN        NaN        NaN   
658717   2823935ML06        NaN        NaN        NaN   
812424   3236169ML06        NaN        NaN        NaN   
1459181  5409272ML06        NaN        NaN        NaN   
87488     7


Exported as table, grouping NER01. Mon Nov 21 14:30:07 2022 

Loaded gridded points for grouping NER02. Mon Nov 21 14:32:14 2022 


Joined MapSPAM 2000 data onto grid ID for grouping NER02. Mon Nov 21 14:32:46 2022 


Joined MapSPAM 2005 data onto grid ID for grouping NER02. Mon Nov 21 14:33:16 2022 


Joined MapSPAM 2010 data onto grid ID for grouping NER02. Mon Nov 21 14:33:47 2022 


Joined MapSPAM 2017 data onto grid ID for grouping NER02. Mon Nov 21 14:34:18 2022 

            GRID_ID  ProdSum00  ProdSum05  ProdSum10  \
25706   5055273NE05        NaN        NaN        NaN   
284300  7055030NE05        NaN   0.001251   0.002945   
330325  7280130NE05        NaN        NaN        NaN   
373958  7487977NE05   0.002828        NaN   0.000005   
185776  6496720NE05        NaN        NaN   0.002205   
345824  7352360NE04   0.000111        NaN   0.006385   
402422  7623736NE05   0.000045   0.008100   0.010392   
447897  7838451NE05        NaN   0.002245   0.000959   
162935  6357488NE05 